In [22]:
import pandas as pd
from json_ntv import Ntv
from datetime import date, time, datetime, timezone, timedelta

In [49]:
types = pd.DataFrame({'ntv_type':  ['json', 'datetime', 'int32', 'int64'], 
                      'name_type': [None, None, None, 'int64'], 
                      'dtype':     [None, 'datetime64[ns]', 'Int32', 'Int64']})

'''type_to_dtype = {'datetime': 'datetime64[ns]',
                 'string': 'string', 'int32': 'int32', 'int64': 'int64',
                 'float': 'float', 'float32': 'float32', 'boolean': 'bool',
                 'json': None}
dtype_to_type = {'datetime64[ns]': 'datetime', 'string': 'string', 
                 'int32': 'int32', 'int64': 'json', 'float': 'json', 
                 'float32': 'float32', 'bool': 'json'}'''

def from_ntv(ntv_value):
    ''' convert ntv_value into the return object'''
    ntv = Ntv.obj(ntv_value)
    ntv_name = ntv.name
    ntv_type = ntv.type_str
    ntv_obj = ntv.to_obj(encoded=True, simpleval=True)
    
    dtype = types.set_index('ntv_type').loc[ntv_type]['dtype']
    name_type = types.set_index('ntv_type').loc[ntv_type]['name_type']
    pd_name = ntv_name + '::' + name_type if name_type else ntv_name
    pd_name = pd_name if pd_name else None
    
    return pd.read_json(ntv_obj, dtype=dtype, typ='series').rename(pd_name)

def to_ntv(self):
    ''' convert object into the NTV entity (name, type, value)'''
    to_type = SeriesConnec.dtype_to_type
    dtype = self.dtype.name
    ntv_type = None
    ntv_name = self.name if self.name else ''
    if dtype in to_type and to_type[dtype] != 'json':
        ntv_type = to_type[dtype]
    if dtype in to_type:
        js = self.to_json(orient='records', date_format='iso', default_handler=str)
        ntv_value = json.loads(js)
    elif dtype == 'object':
        ntv_name, ntv_type, sep = Ntv.from_obj_name(ntv_name)
        ntv_value = self.to_list()
    return (None, 'field', NtvList(ntv_value, ntv_name, ntv_type).to_obj())

In [54]:
a = {'test::int32':[1,2,3]}
a = {'test':[1,2,3]}
a= [1,2.1,3]
a = {'::int64':[1,2,3]}
print(from_ntv(a))


0    1
1    2
2    3
Name: ::int64, dtype: Int64


In [29]:
srs = [pd.Series([1,2,3], dtype='int64'),
       pd.Series([True, False, True], dtype='bool'),
       pd.Series([datetime(2022, 1, 1), datetime(2022, 1, 2)], dtype='datetime64[ns]'),
       pd.Series([{'a': 3, 'e':5}, {'a': 4, 'e':6}]),  
       pd.Series([[3,4], [5,6]]),  
       pd.Series([[3,4], {'a': 3, 'e':5}]),  
       pd.Series([pd.NaT, pd.NaT, pd.NaT]),
       pd.Series([True, False, True]),
       pd.Series(['az', 'er', 'cd']),
       pd.Series([1,2,3]),
       pd.Series([1.1,2,3]),
       
       pd.Series([datetime(2022, 1, 1), datetime(2022, 1, 2)], dtype='datetime64[ns, UTC]'),
       pd.Series([1,2,3], dtype='int32'),
       pd.Series([1,2,3], dtype='UInt64'),
       pd.Series([1,2,3], dtype='float64'),
       pd.Series([1,2,3], dtype='object'),
       pd.Series([(3,4), (5,6)]),  
       pd.Series(['az', 'er', 'cd'], dtype='string'),
       pd.Series([None, None, None]),
       pd.Series([date(2022, 1, 1), date(2022, 1, 2)])
]
for sr in srs:
    print(sr.equals(pd.read_json(sr.to_json(orient='records', date_format='iso', default_handler=str), typ='series')))


True
True
True
True
True
True
True
True
True
True
True
False
False
False
False
False
False
False
False
False


In [25]:
sr = pd.Series([date(2022, 1, 1), date(2022, 1, 2)], dtype='datetime64[ns, CET]')
print(sr)
sr2 = pd.Series([datetime(2022, 1, 1, tzinfo=timezone.utc), datetime(2022, 1, 2, tzinfo=timezone(timedelta(hours=2)))])
print(sr2)
print(type(sr2[1]))
print(sr.to_json(orient='records', date_format='iso', default_handler=str))
print(pd.read_json(sr.to_json(orient='records', date_format='iso', default_handler=str), typ='series'))
print(pd.read_json(sr.to_json(orient='records', default_handler=str), dtype='datetime64[ns, UTC]', typ='series'))


0   2022-01-01 00:00:00+01:00
1   2022-01-02 00:00:00+01:00
dtype: datetime64[ns, CET]
0    2022-01-01 00:00:00+00:00
1    2022-01-02 00:00:00+02:00
dtype: object
<class 'datetime.datetime'>
["2021-12-31T23:00:00.000","2022-01-01T23:00:00.000"]
0   2021-12-31 23:00:00
1   2022-01-01 23:00:00
dtype: datetime64[ns]
0   1970-01-01 00:27:20.991600+00:00
1   1970-01-01 00:27:21.078000+00:00
dtype: datetime64[ns, UTC]


In [35]:
sr = pd.Series([date(2022, 1, 1), date(2022, 1, 2)], dtype='datetime64[ns, CET]', name='test')
print(sr)
js = sr.to_json(orient='records', date_format='iso', default_handler=str)
sr2 = pd.read_json(js, dtype='datetime64[ns, UTC]', typ='series').rename('test2')
print(sr2)

0   2022-01-01 00:00:00+01:00
1   2022-01-02 00:00:00+01:00
Name: test, dtype: datetime64[ns, CET]
0   2021-12-31 23:00:00+00:00
1   2022-01-01 23:00:00+00:00
Name: test2, dtype: datetime64[ns, UTC]


## mapping des types
Type json implicite (pas de ::type):
- json(number) - float64, int64
- json(string) - object(string)
- json(object) - object(dict)
- json(array) - object(tuple)
- json(bool) - bool
- json(null) - NaT

Type pandas spécifique (::type non maintenu sur pandas):
- datetime - datetime64[ns]
- intxx, uintxx - Intxx, UIntxx
- floatxx - floatxx
- string - string

Type pandas explicite (::type maintenu sur pandas):
- float64 - float64
- int64 - Int64
- object - object
- array - object
- bool - bool
- null - NaT

Type  externe (::name maintenu sur pandas)

A voir
- datetime64[ns, <tz>]
- category
- period[<freq>]
- Sparse
- Interval

## conversion 
Type json:
- -> read_json(sans dtype)
- <- to_json

Type pandas spécifique ou explicite:
- -> read_json(avec dtype)
- <- to_json

Type externe:
- -> Ntv.to_obj(format='obj') -> Series
- <- Ntv.from_obj() <- Series

In [39]:
a = pd.read_json('[false,true,null]', typ='series', dtype='Bool' )
print(type(a[0]))
print(type(a[2]))
print(a[0], '\n')
print(a)

<class 'bool'>
<class 'NoneType'>
False 

0    False
1     True
2     None
dtype: object
